# **Hyper Parameter Tuning the ANN Model using Keras Tuner**

In the last notbook i.e ANN_model_Hyper_Parameter_Tuning we used grid serach cv to scheck all the best parameters 

Here we will use a special module called Keras Tuner to decide Number of Hidden Layers And Neuron In Neural Network

Official keras tuner page : https://keras.io/keras_tuner/

To get the more info: https://www.tensorflow.org/tutorials/keras/keras_tuner

In [1]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

<ipython-input-1-63cf4df417c7>:4: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [2]:
df = pd.read_csv('Real_Combine.csv')

In [3]:
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [4]:
df.shape

(1093, 9)

In [5]:
X=df.iloc[:,:-1] ## independent features
y=df.iloc[:,-1] ## dependent features

## Hyperparameters
1. How many number of hidden layers we should have?
2. How many number of neurons we should have in hidden layers?
3. Learning Rate 

In [5]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):  # Number of hidden layers from 2 to 20
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))  # inside that layer how many number of neurons we need from 32 to 512, activation function is relu 
        
    model.add(layers.Dense(1, activation='linear'))  # this is the last output layer, we used linear as activation function for regression
    
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mean_absolute_error',
        metrics=['mean_absolute_error']) # taken adam optimizer, and learning rate is varying from 1e2 to 1e4, and loss is mae and metrix is mae for regression
    
    return model

### here the hyperparameters are:

1) how many number of hidden layers we have
2) how many number of neurons we shoukd have in each hidden layers
3) What is the learning rate

In [6]:
tuner = RandomSearch(
    build_model,
    objective='val_mean_absolute_error',
    max_trials=5,
    executions_per_trial=3,
    directory='project',
    project_name='Air Quality Index')

# here 15 trails it will rul because max trail = 5, executions_per_trial=3

INFO:tensorflow:Reloading Oracle from existing project project\Air Quality Index\oracle.json
INFO:tensorflow:Reloading Tuner from project\Air Quality Index\tuner0.json


In [7]:
tuner.search_space_summary()

Search space summary
Default search space size: 18
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
units_2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_3 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_4 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_5 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_6 (Int)
{'default': None, 'co

In [8]:
# Train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [9]:
tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test))

INFO:tensorflow:Oracle triggered exit


In [10]:
tuner.results_summary()

Results summary
Results in project\Air Quality Index
Showing 10 best trials
Objective(name='val_mean_absolute_error', direction='min')
Trial summary
Hyperparameters:
num_layers: 16
units_0: 192
units_1: 224
learning_rate: 0.01
units_2: 192
units_3: 448
units_4: 352
units_5: 480
units_6: 128
units_7: 192
units_8: 512
units_9: 256
units_10: 64
units_11: 256
units_12: 32
units_13: 32
units_14: 32
units_15: 32
Score: nan
Trial summary
Hyperparameters:
num_layers: 4
units_0: 96
units_1: 192
learning_rate: 0.0001
units_2: 160
units_3: 352
units_4: 416
units_5: 224
units_6: 64
units_7: 416
units_8: 448
units_9: 160
units_10: 64
units_11: 480
Score: nan
Trial summary
Hyperparameters:
num_layers: 12
units_0: 224
units_1: 256
learning_rate: 0.0001
units_2: 32
units_3: 32
units_4: 32
units_5: 32
units_6: 32
units_7: 32
units_8: 32
units_9: 32
units_10: 32
units_11: 32
Score: nan
Trial summary
Hyperparameters:
num_layers: 6
units_0: 128
units_1: 448
learning_rate: 0.0001
units_2: 96
units_3: 224
u